__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
avito_path = 'sevastopol/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-08 13:30:29 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-08 13:30:29 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-08 13:30:32 | AdsListParser: downloader init
[INFO    ] 2022-09-08 13:30:32 | AvitoParser: init
[INFO    ] 2022-09-08 13:30:38 | AvitoParser: 80 pages for read
[INFO    ] 2022-09-08 13:30:38 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-08 13:30:40 | AdsListParser: read page 1
[INFO    ] 2022-09-08 13:30:44 | AdsListParser: read page 2
[INFO    ] 2022-09-08 13:30:47 | AdsListParser: read page 3
[INFO    ] 2022-09-08 13:30:51 | AdsListParser: read page 4
[INFO    ] 2022-09-08 13:30:54 | AdsListParser: read page 5
[INFO    ] 2022-09-08 13:30:57 | AdsListParser: read page 6
[INFO    ] 2022-09-08 13:31:01 | AdsListParser: read page 7
[INFO    ] 2022-09-08 13:31:04 | AdsListParser: read page 8
[INFO    ] 2022-09-08 13:31:07 | AdsListParser: read page 9
[INFO    ] 2022-09-08 13:31:11 | AdsListParser

In [10]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3982
3270


In [11]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [12]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
1455,2482017748,"Объявление «2-к. квартира, 50 м², 1/3 эт.» 11 фотографий",6199000,,"Нахимовский муниципальный округ, посёлок Любимовка, ул. Софьи Перовской, 45","Продаётся двухкомнатная квартира на первом этаже трехэтажного дома в тихом, спальном, курортном районе. Квартира пустая, требует ремонта. Окна металлопластиковые, поменяно отопление (трубы и батареи), проведен газ, имеется отдельное подсобное помещение. Ес",38,2022-09-08 13:36:05.789208
39,2372604864,"Объявление «1-к. квартира, 59 м², 3/3 эт.» 6 фотографий",14245371,ЖК «Голубая Бухта»,"аллея Защитников 35-й Батареи, д. 11а","ЖК «Голубая Бухта» — это неповторимое сочетание современной архитектуры и комфортной жизни с удобным расположением у самого берега Черного моря и живописным видом на Голубую бухту. \nКомплекс имеет уникальную инфраструктуру:300м до пляжа «Голубая бухта», Sp",1,2022-09-08 13:36:05.789208
